In [2]:
# ! pip install sentence-transformers

In [1]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pandas as pd
import os
import json
import re

I0605 14:34:02.738928 4438582720 file_utils.py:35] PyTorch version 1.0.1.post2 available.
/Users/markberger/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/markberger/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/markberger/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

In [23]:
all_papers = pd.read_pickle("../Pickles/all_papers_initial_with_abstract_sents.pkl")

In [24]:
from pprint import pprint

In [2]:


embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

# # Corpus with example sentences
# corpus = ['A man is eating food.',
#           'A man is eating a piece of bread.',
#           'The girl is carrying a baby.',
#           'A man is riding a horse.',
#           'A woman is playing violin.',
#           'Two men pushed carts through the woods.',
#           'A man is riding a white horse on an enclosed ground.',
#           'A monkey is playing drums.',
#           'A cheetah is running behind its prey.',
#           'Hyenas are hunting on an animal.'
#           ]
# corpus_embeddings = embedder.encode(corpus)

# # # Query sentences:
# queries = ['A cheetah chases prey on across a field.']
# query_embeddings = embedder.encode(queries)

# # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
# closest_n = 5
# for query, query_embedding in zip(queries, query_embeddings):
#     distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
    
#     results = zip(range(len(distances)), distances)
#     results = sorted(results, key=lambda x: x[1])

#     print("\n\n======================\n\n")
#     print("Query:", query)
#     print("\nTop 5 most similar sentences in corpus:")

#     for idx, distance in results[0:closest_n]:
#         print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))

I0605 14:34:20.026025 4438582720 SentenceTransformer.py:29] Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
I0605 14:34:20.027601 4438582720 SentenceTransformer.py:32] Did not find a '/' or '\' in the name. Assume to download model from server.
I0605 14:34:20.029026 4438582720 SentenceTransformer.py:68] Load SentenceTransformer from folder: /Users/markberger/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip
I0605 14:34:20.048432 4438582720 configuration_utils.py:182] loading configuration file /Users/markberger/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-base-nli-stsb-mean-tokens.zip/0_RoBERTa/config.json
I0605 14:34:20.049967 4438582720 configuration_utils.py:199] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,


In [26]:
all_papers.sample()

,title,abstract,date,authors,format,subjects,za_id,uri,full_sentences_path,full_scibert_embedding_path,abstract_sents,abstract_and_title_sents
71,Metric-Optimized Example Weights,Real-world machine learning applications oft...,2018-05-27T00:00:00+00:00,"[{'first_name': 'Sen', 'last_name': 'Zhao', 'f...",scientific paper,"[stat.ML, cs.AI, cs.LG]",249ad73104a89386bba1b9c5895f2657c319ec03,https://arxiv.org/abs/1805.10582,../zetaobjects_v1/249a/249ad73104a89386bba1b9c...,../zetaobjects_v1/embeddings/249a/249ad73104a8...,[Real-world machine learning applications ofte...,"[Metric-Optimized Example Weights, Real-world ..."


In [28]:
all_papers.sample(5)[["abstract", "abstract_and_title_sents", "abstract_sents", "title"]]

,abstract,abstract_and_title_sents,abstract_sents,title
391,Deep neural networks have gained tremendous ...,[Unsupervised Representation Learning by Predi...,[Deep neural networks have gained tremendous s...,Unsupervised Representation Learning by Predic...
62,"Many visual surveillance tasks, e.g.video su...",[Learning from Multiple Sources for Video Summ...,"[Many visual surveillance tasks, e.g.video sum...",Learning from Multiple Sources for Video Summa...
265,We develop a new approach that computes appr...,[Computing Strong Game-Theoretic Strategies in...,[We develop a new approach that computes appro...,Computing Strong Game-Theoretic Strategies in ...
155,Texture analysis is an important field of in...,[Image decomposition with anisotropic diffusio...,[Texture analysis is an important field of inv...,Image decomposition with anisotropic diffusion...
35,We explore recurrent encoder multi-decoder n...,[Recurrent Semi-supervised Classification and ...,[We explore recurrent encoder multi-decoder ne...,Recurrent Semi-supervised Classification and C...


In [29]:
all_papers.sample(1)["abstract"].iloc[0]

'  To ensure readability, text is often written and presented with due formatting. These text formatting devices help the writer to effectively convey the narrative. At the same time, these help the readers pick up the structure of the discourse and comprehend the conveyed information. There have been a number of linguistic theories on discourse structure of text. However, these theories only consider unformatted text. Multimedia text contains rich formatting features which can be leveraged for various NLP tasks. In this paper, we study some of these discourse features in multimedia text and what communicative function they fulfil in the context. We examine how these multimedia discourse features can be used to improve an information extraction system. We show that the discourse and text layout features provide information that is complementary to lexical semantic information commonly used for information extraction. As a case study, we use these features to harvest structured subject 

In [45]:
import numpy as np

In [31]:
def get_average_sentences_embedding(sentences, model=embedder):
    embeddings = model.encode(sentences)
    return np.mean(embeddings, axis=0)

In [11]:
sample1 = all_papers.sample(1)["abstract_sents"].iloc[0]
sample2 = all_papers.sample(1)["abstract_sents"].iloc[0]
sample3 = all_papers.sample(1)["abstract_sents"].iloc[0]
sample4 = all_papers.sample(1)["abstract_sents"].iloc[0]

In [12]:
# pprint(sample1)
# print("---------------------")
# pprint(sample2)
# print("---------------------")
# pprint(sample3)
# print("---------------------")
# pprint(sample4)

In [13]:
def preprocess_sentence(sentence):
    s = re.sub(r'\.','',sentence)
    return s

In [14]:
def split_text(text):
    initial_split = text.split(".")
    return [preprocess_sentence(s) for s in initial_split if s != ""]

In [43]:
music_sample1 = split_text("The Belyayev circle believed in a national style of classical music, based on the achievements of the composer group The Five which preceded it. One important difference between composers in the Belyayev circle and their counterparts in the Five was an acceptance in the necessity of Western-styled academic training; this was an attitude passed down by Rimsky-Korsakov, who taught many of the composers in the circle at the Saint Petersburg Conservatory. While these composers were more open to Western compositional practices and influences, especially through the music of Pyotr Ilyich Tchaikovsky, they closely followed many of the compositional practices of the Five to the point of mannerism, especially in their depiction of folkloric subject matter.")
tribe_sample1 = split_text("They are known for both their prowess in warfare and trophy-keeping practices, as well as their ability to interact and accommodate non-native peoples. They maintained a nomadic existence and frequently intermarried with other tribes.")
tribe_sample2 = split_text("The Tupi people inhabited almost all of Brazil's coast when the Portuguese first arrived there. In 1500, their population was estimated at 1 million people, nearly equal to the population of Portugal at the time. They were divided into tribes, each tribe numbering from 300 to 2,000 people.")
nationality_sample3 = split_text("The ideologies associated with (Romantic) Nationalism of the 19th and 20th centuries never really caught on in the Netherlands, and this, together with being a relatively mono-ethnic society up until the late 1950s, has led to a relatively obscure use of the terms nation and ethnicity as both were largely overlapping in practice. ")
music_sample2 = split_text("Tchaikovsky displayed a wide stylistic and emotional range, from light salon works to grand symphonies. Some of his works, such as the Variations on a Rococo Theme, employ a Classical form reminiscent of 18th-century composers such as Mozart (his favorite composer).")
company_example1 = split_text("The four networks are often grouped together for a number of reasons; they are each comparable in size relative to the rest of the market, both in terms of revenue and workforce; they are each considered equal in their ability to provide a wide scope of quality professional services to their clients; and, among those looking to start a career in professional services, particularly accounting, they are considered equally attractive networks to work in, because of the frequency with which these firms engage with Fortune 500 companies.")
company_example2 = split_text("Google LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, search engine, cloud computing, software, and hardware.")

In [44]:
music_sample1

['The Belyayev circle believed in a national style of classical music, based on the achievements of the composer group The Five which preceded it',
 ' One important difference between composers in the Belyayev circle and their counterparts in the Five was an acceptance in the necessity of Western-styled academic training; this was an attitude passed down by Rimsky-Korsakov, who taught many of the composers in the circle at the Saint Petersburg Conservatory',
 ' While these composers were more open to Western compositional practices and influences, especially through the music of Pyotr Ilyich Tchaikovsky, they closely followed many of the compositional practices of the Five to the point of mannerism, especially in their depiction of folkloric subject matter']

In [45]:
corpus = [music_sample1, tribe_sample1, tribe_sample2, nationality_sample3, music_sample2, company_example1, company_example2]

In [46]:
corpus_embeddings = [get_average_sentences_embedding(sents) for sents in corpus]

In [47]:
# # Query sentences:
queries = ['Large technology companies earn way too much money.', 'The classical music is very influential on a lot of art movements.', 'There are multiple nationalitites present in this rural country.']
query_embeddings = embedder.encode(queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
    
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(' '.join(corpus[idx]), "(Score: %.4f)" % (1-distance))





Query: Large technology companies earn way too much money.

Top 5 most similar sentences in corpus:
The four networks are often grouped together for a number of reasons; they are each comparable in size relative to the rest of the market, both in terms of revenue and workforce; they are each considered equal in their ability to provide a wide scope of quality professional services to their clients; and, among those looking to start a career in professional services, particularly accounting, they are considered equally attractive networks to work in, because of the frequency with which these firms engage with Fortune 500 companies (Score: 0.3144)
Google LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, search engine, cloud computing, software, and hardware (Score: 0.2392)
Tchaikovsky displayed a wide stylistic and emotional range, from light salon works to grand symphonies  S

In [38]:
similarity = 1 - scipy.spatial.distance.cdist([query_embeddings[0]], [corpus_embeddings[6]], "cosine")[0]
print(similarity)

[0.13890238]


# Save abstract average SBERT embedding + title SBERT embedding to npy

In [2]:
all_papers.sample(5)

NameError: name 'all_papers' is not defined

In [56]:
abstracts = [[2,3,1], [4,5,2]]
title = [7,1,2]
abstracts_and_title = [[7,1,2], [2,3,1], [4,5,2]]

In [60]:
np.mean(abstracts, axis=0)

array([3. , 4. , 1.5])

In [58]:
np.mean(abstracts_and_title, axis=0)

array([4.33333333, 3.        , 1.66666667])

In [59]:
np.mean([np.mean(abstracts, axis=0), title], axis=0)

array([5.  , 2.5 , 1.75])

In [63]:
13/3

4.333333333333333

In [64]:
10/2

5.0

In [47]:
def calculate_average_embedding_abstract_and_title(abstract, title, strategy="merge", model=embedder):

    title_embedding = model.encode(title)
    
    if strategy == "merge":
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
    elif strategy == "separate":
        average_abstract = get_average_sentences_embedding(abstract)
        average = np.mean([average_abstract, title_embedding[0]], axis=0)
    else:
        print("Warning: wrong strategy is used. Use either 'merge' or 'separate'. Proceeding using the 'merge' strategy.")
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
        
    return average
        

In [48]:
def store_abstract_sbert_embedding(row, emb_strategy="merge"):
    title = row["title"]
    abstract = row["abstract_sents"]
    za_id = row["za_id"]
    za_id_short = za_id[:4]
    
    path_to_sbert_representation = row["full_sentences_path"][:-22]
    
    folder_merge = row["full_scibert_embedding_path"].replace("zetaobjects_v1/embeddings/", "zetaobjects_v1/sbert_embedings_abstract_title_merged/").replace("/document_embedding.npy", "")
    folder_separate = row["full_scibert_embedding_path"].replace("zetaobjects_v1/embeddings/", "zetaobjects_v1/sbert_embedings_abstract_title_separate/").replace("/document_embedding.npy", "")
    embedding = calculate_average_embedding_abstract_and_title(abstract, title, strategy=emb_strategy)
    if emb_strategy=="merge":
        if not os.path.exists(folder_merge):
            os.makedirs(folder_merge)
        np.save(f"{folder_merge}/embedding.npy", embedding)
            
        with open(f"{path_to_sbert_representation}sbert_embedding_merged", 'w', encoding='utf-8') as f:
            json.dump({"version":"1.0.0","value":{"pointer":f"/data/zetaobjects_v1/sbert_embedings_abstract_title_merged/{za_id_short}/{za_id}/embedding.npy"},"type":"pointer"}, f, ensure_ascii=False, indent=4)
        
    elif emb_strategy=="separate":
        if not os.path.exists(folder_separate):
            os.makedirs(folder_separate)
        np.save(f"{folder_separate}/embedding.npy", embedding) 
        
        with open(f"{path_to_sbert_representation}sbert_embedding_separate", 'w', encoding='utf-8') as f:
            json.dump({"version":"1.0.0","value":{"pointer":f"/data/zetaobjects_v1/sbert_embedings_abstract_title_separate/{za_id_short}/{za_id}/embedding.npy"},"type":"pointer"}, f, ensure_ascii=False, indent=4)

In [51]:
# for index, row in all_papers.iterrows():
#     store_abstract_sbert_embedding(row, "merge")